In [6]:
# 原始声音文件路径
__CORPUSPATH__ = "voice"

# 处理后的路径
__OUTPATH__ = "data"

In [7]:
from pydub import AudioSegment
from pydub.silence import split_on_silence
import os

def split(sound):
    dBFS = sound.dBFS
    chunks = split_on_silence(sound,
        min_silence_len = 100,
        silence_thresh = dBFS-16,
        keep_silence = 100
    )
    return chunks

def combine(_src):
    audio = AudioSegment.empty()
    for i,filename in enumerate(os.listdir(_src)):
        if filename.endswith('.wav'):
            filename = os.path.join(_src, filename)
            audio += AudioSegment.from_wav(filename)
    return audio

def save_chunks(chunks, directory):
    if not os.path.exists(directory):
        os.makedirs(directory)
    counter = 0

    target_length = 5 * 1000
    output_chunks = [chunks[0]]
    for chunk in chunks[1:]:
        if len(output_chunks[-1]) < target_length:
            output_chunks[-1] += chunk
        else:
            # if the last output chunk is longer than the target length,
            # we can start a new one
            output_chunks.append(chunk)

    for chunk in output_chunks:
        chunk = chunk.set_frame_rate(24000)
        chunk = chunk.set_channels(1)
        counter = counter + 1
        chunk.export(os.path.join(directory, str(counter) + '.wav'), format="wav")

In [9]:
# downsample to 24 kHz
# 直接在这里运行即可把所有的音频转换
for p in range(1,21):
    directory = __OUTPATH__ + '/' + str(p)
    if not os.path.exists(directory):
        audio = combine(__CORPUSPATH__ + '/' + str(p))
        chunks = split(audio)
        save_chunks(chunks, directory)